In [1]:
from string import ascii_lowercase as ALPHABET
import numpy as np
import numpy.random as r
import pandas as pd

In [2]:
THOUSAND_NOUNS = [x for x in pd.read_csv('most-common-nouns-english.csv')['Word'].values]

In [16]:
class WordSearchGrid:
    def __init__(self, *, size, seed):
        r.seed(seed)
        self.size = size
        self.grid = np.char.array([['.']*self.size] * self.size)
        self.words = []
        
    def __repr__(self):
        repr = ""
        for line in self.grid:
            repr = repr + ' '.join([x for x in line]) + '\n'
        return repr
    
    def try_add_word(self, word, max_tries=100):
        if len(word) >= self.size:
            return False
        
        actions = [
            self.try_add_forward_horizontal_word,
            self.try_add_forward_vertical_word,
            self.try_add_reverse_horizontal_word,
            self.try_add_reverse_vertical_word,
        ]
        
        tries = 0
        while(tries < max_tries):
            action = r.choice(actions)
            if action(word):
                self.words.append(word)
                return True
            tries += 1
            
        return False
    
    def try_add_reverse_vertical_word(self, word):
        row = r.randint(0,self.size-len(word))
        column = r.randint(0,self.size)

        slot = self.grid[row:row+len(word), column]
        if not all(slot=='.'):
            return False
        
        for index, letter in enumerate(word):
            self.grid[row+(len(word) -1 - index), column] = letter
        return True
    
    def try_add_forward_vertical_word(self, word):
        row = r.randint(0,self.size-len(word))
        column = r.randint(0,self.size)

        slot = self.grid[row:row+len(word), column]
        if not all(slot=='.'):
            return False
        
        for index, letter in enumerate(word):
            self.grid[row+index, column] = letter
        return True
    
    def try_add_forward_horizontal_word(self, word):
        row = r.randint(0,self.size)
        column = r.randint(0,self.size-len(word))

        slot = self.grid[row, column: column+len(word)]
        if not all(slot=='.'):
            return False
        
        for index, letter in enumerate(word):
            self.grid[row, column+index] = letter
        return True
    
    def try_add_reverse_horizontal_word(self, word):
        row = r.randint(0,self.size)
        column = r.randint(0,self.size-len(word))

        slot = self.grid[row, column: column+len(word)]
        if not all(slot=='.'):
            return False
        
        for index, letter in enumerate(word):
            self.grid[row, column+(len(word) -1 - index)] = letter
        return True
        
    def fill_empty(self):
        letters = []
        for word in self.words:
            for letter in word:
                letters.append(letter)
        
        letters = sorted([letter for letter in sorted(letters)])
        
        for i, line in enumerate(self.grid):
            for j, letter in enumerate(line):
                if letter == '.':
                    self.grid[i,j] = r.choice([x for x in ALPHABET])
                    
                    
    def fill(self, words):
        for word in words:
            self.try_add_word(word)
        self.fill_empty()
        self.words = sorted(self.words)
        return self

In [17]:
def GenerateWordSearch(*, size, seed, word_try_count, dictionary=THOUSAND_NOUNS):
    words = r.choice([x for x in dictionary if len(x)<=size and len(x)>4], size=word_try_count, replace=False)
    w = WordSearchGrid(size=size, seed=seed).fill(words)
    return """Wordsearch #{seed}
    
{word_list}

{grid}""".format(seed=seed, word_list='\n'.join(f'[ ] {w}' for w in w.words), grid=w)


In [18]:
for x in range(100):
    with open(f'/tmp/wordsearch_{x}.txt','w') as o:
        o.write(GenerateWordSearch(size=20, seed=x, word_try_count=25, dictionary=THOUSAND_NOUNS))